## This notebook records some of the tricks I have learnt from work and study with Python

## String manipulation

### Example 1: How to read a string column, extract certain pattern in each row (within that column) and create a new column

In [1]:
import pandas as pd
import numpy as np

In [2]:
P_df = pd.DataFrame({'VEH_INFO' : ['Coupe XP 4S','Sedan XLT 4cylinder','PICKUP EXTRA','SEDAN XL LT','COUPE 4_cyl SE','pickup XST','Wagon SET 4wd'], 'Value':[100,200,300,400,500,600,700]})
P_df

,VEH_INFO,Value
0,Coupe XP 4S,100
1,Sedan XLT 4cylinder,200
2,PICKUP EXTRA,300
3,SEDAN XL LT,400
4,COUPE 4_cyl SE,500
5,pickup XST,600
6,Wagon SET 4wd,700


In [3]:
# I wanted to find out the vehicle style of each obervation, either coupe, or sedan or pickup or something else


In [4]:
# Define a function to identify if a keyword is contained:
def VEH_STYLE_FN(DAT):
    DAT = pd.Series(DAT)
    if int(DAT.str.contains('COUPE')) ==1:
        return('COUPE')
    if int(DAT.str.contains('PICKUP')) ==1:
        return('PICKUP')
    if int(DAT.str.contains('SEDAN')) ==1:
        return('SEDAN')
    else: return('OTHERS')



In [5]:
P_df['STYLE'] = P_df['VEH_INFO'].str.split(' ').apply(lambda x: str(x).upper()).apply(lambda x: VEH_STYLE_FN(x))
P_df

,VEH_INFO,Value,STYLE
0,Coupe XP 4S,100,COUPE
1,Sedan XLT 4cylinder,200,SEDAN
2,PICKUP EXTRA,300,PICKUP
3,SEDAN XL LT,400,SEDAN
4,COUPE 4_cyl SE,500,COUPE
5,pickup XST,600,PICKUP
6,Wagon SET 4wd,700,OTHERS


### Example 2: Another example of extracting information from one string column and assign value to a new column

In [6]:
# Let's add more columns to P_df:
P_df_ex2 = P_df.copy()
P_df_ex2['VIN'] = ['2T1BU12348', '1F1BU45613','2T1BU12789','ZZ1BU1206Z','2T1BU12832','2T1BU12806','2T1BU12555']
P_df_ex2

,VEH_INFO,Value,STYLE,VIN
0,Coupe XP 4S,100,COUPE,2T1BU12348
1,Sedan XLT 4cylinder,200,SEDAN,1F1BU45613
2,PICKUP EXTRA,300,PICKUP,2T1BU12789
3,SEDAN XL LT,400,SEDAN,ZZ1BU1206Z
4,COUPE 4_cyl SE,500,COUPE,2T1BU12832
5,pickup XST,600,PICKUP,2T1BU12806
6,Wagon SET 4wd,700,OTHERS,2T1BU12555


In [7]:
# Define a function to allocate the vehicle to grps based on the last two digits in the 'fake' VIN

## Here is the logic: 
## If the second to last digit is 0,1 --> Ctrl group
## For the rest vehicles, if last digit is 0-3 --> Group 1, if 4-6 --> Group 2, else, Group 3

def VIN_random_fn(DAT):
    vin = DAT
    if int(vin[-2]) in [0,1]:
        output = 'Ctrl'
    elif int(vin[-2]) in [2,3,4,5,6,7,8,9]:
        if int(vin[-1]) in [0,1,2,3]:
            output = 'Grp_1'
        elif int(vin[-1]) in [4,5,6]:
            output = 'Grp_2'
        else:
            output = 'Grp_3'
    return output

In [8]:
# First of all, we wanted to eliminate if the last digit is alphabet, not a number:

P_df_ex2 = P_df_ex2[P_df_ex2['VIN'].astype(str).map(lambda dat: dat[-1] in ['1','2','3','4','5','6','7','8','9','0'])].copy()

In [9]:
# Then, let's create a new column called "Test_cell" for grouping:

P_df_ex2['Test_cell'] = P_df_ex2['VIN'].astype(str).apply(lambda dat: VIN_random_fn(dat))

In [10]:
P_df_ex2

,VEH_INFO,Value,STYLE,VIN,Test_cell
0,Coupe XP 4S,100,COUPE,2T1BU12348,Grp_3
1,Sedan XLT 4cylinder,200,SEDAN,1F1BU45613,Ctrl
2,PICKUP EXTRA,300,PICKUP,2T1BU12789,Grp_3
4,COUPE 4_cyl SE,500,COUPE,2T1BU12832,Grp_1
5,pickup XST,600,PICKUP,2T1BU12806,Ctrl
6,Wagon SET 4wd,700,OTHERS,2T1BU12555,Grp_2


### Example 3: Wanted to cap the values in a certain column
For example, in the P_df_ex2, Values column has values ranging from 100 to 700. Let's cap it at 550

In [11]:
# First, let's do an copy on the P_df_ex2
P_df_ex3 = P_df_ex2.copy()

# then, use the where function:
P_df_ex3['Value_capped_byWhere'] = P_df_ex3['Value'].where(P_df_ex3['Value'] < 550, 550)

# Print the result
P_df_ex3

,VEH_INFO,Value,STYLE,VIN,Test_cell,Value_capped_byWhere
0,Coupe XP 4S,100,COUPE,2T1BU12348,Grp_3,100
1,Sedan XLT 4cylinder,200,SEDAN,1F1BU45613,Ctrl,200
2,PICKUP EXTRA,300,PICKUP,2T1BU12789,Grp_3,300
4,COUPE 4_cyl SE,500,COUPE,2T1BU12832,Grp_1,500
5,pickup XST,600,PICKUP,2T1BU12806,Ctrl,550
6,Wagon SET 4wd,700,OTHERS,2T1BU12555,Grp_2,550


It is also possible to write a customed function and use apply, Let's try:

In [12]:
# First, define a function:

def cap_fun(x):
    if x > 500:
        new_x = 550
    else: new_x = x
    return new_x

# then, let's apply this function to dataframe
P_df_ex3['Value_capped_byApply_1'] = P_df_ex3.apply(lambda dat: cap_fun(dat['Value']),axis=1)


# alternative way to use apply
P_df_ex3['Value_capped_byApply_2'] = P_df_ex3['Value'].apply(lambda dat: cap_fun(dat))

# Print results
P_df_ex3

,VEH_INFO,Value,STYLE,VIN,Test_cell,Value_capped_byWhere,Value_capped_byApply_1,Value_capped_byApply_2
0,Coupe XP 4S,100,COUPE,2T1BU12348,Grp_3,100,100,100
1,Sedan XLT 4cylinder,200,SEDAN,1F1BU45613,Ctrl,200,200,200
2,PICKUP EXTRA,300,PICKUP,2T1BU12789,Grp_3,300,300,300
4,COUPE 4_cyl SE,500,COUPE,2T1BU12832,Grp_1,500,500,500
5,pickup XST,600,PICKUP,2T1BU12806,Ctrl,550,550,550
6,Wagon SET 4wd,700,OTHERS,2T1BU12555,Grp_2,550,550,550


### Example 4: groupby,count and make a summary table

In [13]:
# Copy the dataframe
P_df_ex4 = P_df_ex3.copy()

# Manually create a duplicated observations in the df
P_df_ex4 = P_df_ex4.append(P_df_ex4, ignore_index = True).copy()

# Print results
P_df_ex4


,VEH_INFO,Value,STYLE,VIN,Test_cell,Value_capped_byWhere,Value_capped_byApply_1,Value_capped_byApply_2
0,Coupe XP 4S,100,COUPE,2T1BU12348,Grp_3,100,100,100
1,Sedan XLT 4cylinder,200,SEDAN,1F1BU45613,Ctrl,200,200,200
2,PICKUP EXTRA,300,PICKUP,2T1BU12789,Grp_3,300,300,300
3,COUPE 4_cyl SE,500,COUPE,2T1BU12832,Grp_1,500,500,500
4,pickup XST,600,PICKUP,2T1BU12806,Ctrl,550,550,550
5,Wagon SET 4wd,700,OTHERS,2T1BU12555,Grp_2,550,550,550
6,Coupe XP 4S,100,COUPE,2T1BU12348,Grp_3,100,100,100
7,Sedan XLT 4cylinder,200,SEDAN,1F1BU45613,Ctrl,200,200,200
8,PICKUP EXTRA,300,PICKUP,2T1BU12789,Grp_3,300,300,300
9,COUPE 4_cyl SE,500,COUPE,2T1BU12832,Grp_1,500,500,500


In [14]:
# Group, count and make a summary table:
P_df_ex4.groupby('STYLE')['VIN'].count().to_frame(name='counts').reset_index()

,STYLE,counts
0,COUPE,4
1,OTHERS,2
2,PICKUP,4
3,SEDAN,2


A very important note here. P_df_ex4 is a duplication of 2 P_df_ex3. But after group by STYLE, the count() function does not care about the duplicated VINs. Bascially it just tells us how many rows are there in each group

### Example 5: Extra the first element in cell in a column in a df, and then copy it to another new column

In [15]:
P_df_ex5 = P_df.copy()
P_df_ex5['Make_Model_Trim'] = ['VW Passat XLE', 'TOYOTA CAMRY SE','Subaru Forester Premium',
                              'Honda CIVIC S', 'CHEVY Malibu LE', 'ACURA RDX XLE', 'Audi Q7 i7' ]
P_df_ex5

,VEH_INFO,Value,STYLE,Make_Model_Trim
0,Coupe XP 4S,100,COUPE,VW Passat XLE
1,Sedan XLT 4cylinder,200,SEDAN,TOYOTA CAMRY SE
2,PICKUP EXTRA,300,PICKUP,Subaru Forester Premium
3,SEDAN XL LT,400,SEDAN,Honda CIVIC S
4,COUPE 4_cyl SE,500,COUPE,CHEVY Malibu LE
5,pickup XST,600,PICKUP,ACURA RDX XLE
6,Wagon SET 4wd,700,OTHERS,Audi Q7 i7


In [16]:
# I wanted to extract the make model trim information from the column "Make_Model_Trim"

P_df_ex5['Make_NM'] = P_df_ex5['Make_Model_Trim'].apply(lambda x: str(x).upper()).apply(lambda x: str(x).split(' ')[0])
P_df_ex5['Model_NM'] = P_df_ex5['Make_Model_Trim'].apply(lambda x: str(x).upper()).apply(lambda x: str(x).split(' ')[1])
P_df_ex5['Trim_NM'] = P_df_ex5['Make_Model_Trim'].apply(lambda x: str(x).split(' ')[2])
P_df_ex5


,VEH_INFO,Value,STYLE,Make_Model_Trim,Make_NM,Model_NM,Trim_NM
0,Coupe XP 4S,100,COUPE,VW Passat XLE,VW,PASSAT,XLE
1,Sedan XLT 4cylinder,200,SEDAN,TOYOTA CAMRY SE,TOYOTA,CAMRY,SE
2,PICKUP EXTRA,300,PICKUP,Subaru Forester Premium,SUBARU,FORESTER,Premium
3,SEDAN XL LT,400,SEDAN,Honda CIVIC S,HONDA,CIVIC,S
4,COUPE 4_cyl SE,500,COUPE,CHEVY Malibu LE,CHEVY,MALIBU,LE
5,pickup XST,600,PICKUP,ACURA RDX XLE,ACURA,RDX,XLE
6,Wagon SET 4wd,700,OTHERS,Audi Q7 i7,AUDI,Q7,i7


In [17]:
# There are some nick nnames in the make info, such as the volkswagon is coded as vw, Chevrolet is coded as chevy.
# The next step is to replace these abbreviations with their original name

make_dict = {
    "CHEVY": "CHEROLET",
    "VW": "VOLKSWAGON"
}
P_df_ex5['Make_NM'] = P_df_ex5['Make_NM'].replace(make_dict)
P_df_ex5

,VEH_INFO,Value,STYLE,Make_Model_Trim,Make_NM,Model_NM,Trim_NM
0,Coupe XP 4S,100,COUPE,VW Passat XLE,VOLKSWAGON,PASSAT,XLE
1,Sedan XLT 4cylinder,200,SEDAN,TOYOTA CAMRY SE,TOYOTA,CAMRY,SE
2,PICKUP EXTRA,300,PICKUP,Subaru Forester Premium,SUBARU,FORESTER,Premium
3,SEDAN XL LT,400,SEDAN,Honda CIVIC S,HONDA,CIVIC,S
4,COUPE 4_cyl SE,500,COUPE,CHEVY Malibu LE,CHEROLET,MALIBU,LE
5,pickup XST,600,PICKUP,ACURA RDX XLE,ACURA,RDX,XLE
6,Wagon SET 4wd,700,OTHERS,Audi Q7 i7,AUDI,Q7,i7


## Time Variable Issue

Time variable sometimes is tricky to handle. So let's take some examples

### Basic codes and handling bi-week indicator

In [18]:
# Create the dataframe to handle:

Time_df = P_df_ex2.copy()
Time_df['DT'] = ['2020-01-21','2020-02-25','2021-03-01','2021-08-18','2022-02-28','2022-05-08']
Time_df

,VEH_INFO,Value,STYLE,VIN,Test_cell,DT
0,Coupe XP 4S,100,COUPE,2T1BU12348,Grp_3,2020-01-21
1,Sedan XLT 4cylinder,200,SEDAN,1F1BU45613,Ctrl,2020-02-25
2,PICKUP EXTRA,300,PICKUP,2T1BU12789,Grp_3,2021-03-01
4,COUPE 4_cyl SE,500,COUPE,2T1BU12832,Grp_1,2021-08-18
5,pickup XST,600,PICKUP,2T1BU12806,Ctrl,2022-02-28
6,Wagon SET 4wd,700,OTHERS,2T1BU12555,Grp_2,2022-05-08


In [19]:
# Basic codes:

## After the running the following codes: 'year' and 'month' will reflect the actual date of the observations
## week is the number of week in that particular year. e.g. 2021-08-18 is in the 33rd week of 2021
## Year_Month is still a time variable showing the date in terms of Year-Month 
## biweek is the number of bi-weeks since the first day of observation (in this case: 2020-01-21). e.g. for 2021-08-18, it is 2*33 = 66 weeks away from 2020-01-21

Time_df['DT'] = pd.to_datetime(Time_df['DT'], format = '%Y-%m-%d')
Time_df['year'] = Time_df['DT'].dt.strftime('%Y').astype(int)
Time_df['month'] = Time_df['DT'].dt.strftime('%m').astype(int)
Time_df['day'] = Time_df['DT'].dt.strftime('%d').astype(int)
Time_df['week'] = Time_df['DT'].dt.strftime('%V').astype(int)

Time_df['Year_Month'] = Time_df['DT'].dt.strftime('%Y-%m')

Time_df['biweek'] = ((Time_df['DT'] - Time_df['DT'].min()).dt.days // 14 +1).astype(int)

Time_df

,VEH_INFO,Value,STYLE,VIN,Test_cell,DT,year,month,day,week,Year_Month,biweek
0,Coupe XP 4S,100,COUPE,2T1BU12348,Grp_3,2020-01-21,2020,1,21,4,2020-01,1
1,Sedan XLT 4cylinder,200,SEDAN,1F1BU45613,Ctrl,2020-02-25,2020,2,25,9,2020-02,3
2,PICKUP EXTRA,300,PICKUP,2T1BU12789,Grp_3,2021-03-01,2021,3,1,9,2021-03,29
4,COUPE 4_cyl SE,500,COUPE,2T1BU12832,Grp_1,2021-08-18,2021,8,18,33,2021-08,42
5,pickup XST,600,PICKUP,2T1BU12806,Ctrl,2022-02-28,2022,2,28,9,2022-02,55
6,Wagon SET 4wd,700,OTHERS,2T1BU12555,Grp_2,2022-05-08,2022,5,8,18,2022-05,60


In [20]:
Time_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6 entries, 0 to 6
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   VEH_INFO    6 non-null      object        
 1   Value       6 non-null      int64         
 2   STYLE       6 non-null      object        
 3   VIN         6 non-null      object        
 4   Test_cell   6 non-null      object        
 5   DT          6 non-null      datetime64[ns]
 6   year        6 non-null      int64         
 7   month       6 non-null      int64         
 8   day         6 non-null      int64         
 9   week        6 non-null      int64         
 10  Year_Month  6 non-null      object        
 11  biweek      6 non-null      int64         
dtypes: datetime64[ns](1), int64(6), object(5)
memory usage: 624.0+ bytes


## Map, Filter, and Reduce

### Map: Using the map function, we can apply a function to each value of an iterable object(list, tuple, etc. ). 
map(function, iterable) 

In [21]:
input_list = [2, 3, 4, 5, 6]

# Without lambda 
def square(x):
    return x*x

result = map(square, input_list)

print(list(result))

# Using lambda function 
result2 = map(lambda x: x*x, input_list)

print(list(result2))


[4, 9, 16, 25, 36]
[4, 9, 16, 25, 36]


### Filter: the filter function is used to filter out values from an iterable object (list, tuple, sets, etc.). The filtering conditions are set inside a function which is passed as an argument to the filter function.
filter(function, iterable)

In [22]:
input_list = [2, 3, 4, 5, 10, 12, 14] 

# Without lambda
def less_than_10(x):
    if x <= 6:
        return x 

print(list(filter(less_than_10, input_list)))

# using lambda function 
print(list(filter(lambda x: x <= 6, input_list)))


[2, 3, 4, 5]
[2, 3, 4, 5]


### Reduce is applied iteratively to all the values of the iterable object and returns only one value.
reduce(function, iterable)

In [23]:
from functools import reduce

input_list = [1, 2, 3, 4, 5]

reduce(lambda x,y: x+y, input_list)

15